In [1]:
!pip install pandas numpy scikit-learn

In [212]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [213]:
movies=pd.read_csv('tmdb_5000_movies.csv')
credits=pd.read_csv('tmdb_5000_credits.csv')

In [214]:
movies=pd.merge(movies,credits,on='title')

In [215]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [216]:
# id,genres,keywords,overview,cast,crew
temp=movies[['movie_id','genres','overview','keywords','title','cast','crew']]
movies=temp

In [217]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4809 non-null   int64 
 1   genres    4809 non-null   object
 2   overview  4806 non-null   object
 3   keywords  4809 non-null   object
 4   title     4809 non-null   object
 5   cast      4809 non-null   object
 6   crew      4809 non-null   object
dtypes: int64(1), object(6)
memory usage: 263.1+ KB


In [218]:
movies.isnull().sum()

movie_id    0
genres      0
overview    3
keywords    0
title       0
cast        0
crew        0
dtype: int64

In [219]:
movies.dropna(inplace=True)

In [220]:
movies.duplicated().sum()

0

In [221]:
import ast
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L


In [222]:
movies['keywords']=movies['keywords'].apply(convert)
movies['genres']=movies['genres'].apply(convert)


In [223]:
movies['genres'][0]

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [224]:
def convert2(obj):
    L=[]
    count=0
    for i in ast.literal_eval(obj):
        if count<3:
            L.append(i['name'])
            count+=1
        else:
            break
    return L


In [225]:
movies['cast']=movies['cast'].apply(convert2)

In [226]:
def convert3(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L

In [227]:
movies['crew']=movies['crew'].apply(convert3)

In [228]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [229]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [230]:
movies['crew']=movies['crew'].apply(lambda x: [i.replace(" ","") for i in x])

In [231]:
movies['cast']=movies['cast'].apply(lambda x: [i.replace(" ","") for i in x])

In [232]:
movies['keywords']=movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x])

In [233]:
movies['tags']=movies['genres']+movies['keywords']+movies['overview']+movies['crew']+movies['cast']
movies_df=movies[['movie_id','title','tags']]

In [234]:
movies_df['tags']=movies_df['tags'].apply(lambda x: " ".join(x))

C:\Users\HP\AppData\Local\Temp\ipykernel_6124\484473674.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['tags']=movies_df['tags'].apply(lambda x: " ".join(x))


In [260]:
movies_df['tags']

0       Action Adventure Fantasy ScienceFiction cultur...
1       Adventure Fantasy Action ocean drugabuse exoti...
2       Action Adventure Crime spy basedonnovel secret...
3       Action Crime Drama Thriller dccomics crimefigh...
4       Action Adventure ScienceFiction basedonnovel m...
                              ...                        
4804    Action Crime Thriller unitedstates–mexicobarri...
4805    Comedy Romance A newlywed couple's honeymoon i...
4806    Comedy Drama Romance TVMovie date loveatfirsts...
4807    When ambitious New York attorney Sam is sent t...
4808    Documentary obsession camcorder crush dreamgir...
Name: tags, Length: 4806, dtype: object

In [5]:
# Vectorization

In [262]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')
    

In [292]:
vectors=cv.fit_transform(movies_df['tags']).toarray()

In [302]:
!pip install nltk

In [304]:
import nltk
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [322]:
def stem(text):
    s=[]
    for i in text.split():
        s.append(ps.stem(i))

    return " ".join(s) 

In [326]:
movies_df['tags']=movies_df['tags'].apply(stem)

C:\Users\HP\AppData\Local\Temp\ipykernel_6124\3732559203.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['tags']=movies_df['tags'].apply(stem)


In [332]:
movies_df['tags']

0       action adventur fantasi sciencefict culturecla...
1       adventur fantasi action ocean drugabus exotici...
2       action adventur crime spi basedonnovel secreta...
3       action crime drama thriller dccomic crimefight...
4       action adventur sciencefict basedonnovel mar m...
                              ...                        
4804    action crime thriller unitedstates–mexicobarri...
4805    comedi romanc a newlyw couple' honeymoon is up...
4806    comedi drama romanc tvmovi date loveatfirstsig...
4807    when ambiti new york attorney sam is sent to s...
4808    documentari obsess camcord crush dreamgirl eve...
Name: tags, Length: 4806, dtype: object

In [338]:
from sklearn.metrics.pairwise import cosine_similarity

In [346]:
similarity=cosine_similarity(vectors)

In [350]:
similarity[0]

array([1.        , 0.08964215, 0.05976143, ..., 0.02519763, 0.02817181,
       0.        ])

In [422]:
def recommend(movie):
    movie_index=movies_df[movies_df['title']==movie].index[0]
    distances=similarity[movie_index]
    movie_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]

    for i in movie_list:
        print(movies_df.iloc[i[0]]['title'])
    

In [424]:
recommend('Avatar')

Titan A.E.
Small Soldiers
Independence Day
Ender's Game
Aliens vs Predator: Requiem
